## Step - 1
### spplitting the dataset into training and testing

In [13]:
import json
import random

# Set a fixed seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Load dataset from a JSON file
with open(r"dataset\high_school_physics.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Shuffle the dataset deterministically
random.shuffle(data)

# Split into 85% train and 15% test
train_size = int(0.85 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

# Save train and test sets
with open(r"dataset\train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4)

with open(r"dataset\test.json", "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4)

print(f"Dataset split into {len(train_data)} training and {len(test_data)} testing samples.")


Dataset split into 340 training and 60 testing samples.


In [14]:
print(test_data[0])
print(test_data[3])

{'id': 151, 'question': 'A ball is thrown vertically upwards with an initial velocity of 35 m/s. What is its velocity at the highest point?', 'subject': 'Kinematics', 'choices': ['0 m/s', '5 m/s', '10 m/s', '-35 m/s'], 'answer': 'A', 'explanation': 'At the highest point, the ball momentarily comes to rest before falling back down.', 'dataset': 'high_school_physics'}
{'id': 194, 'question': 'A convex lens has a focal length of 10 cm. An object is placed 5 cm away. What is the nature of the image?', 'subject': 'Optics', 'choices': ['Real and inverted', 'Real and erect', 'Virtual and inverted', 'Virtual and erect'], 'answer': 'D', 'explanation': 'For a convex lens, when the object is placed between the lens and the focal point, the image is virtual and erect.', 'dataset': 'high_school_physics'}


### Few-Shot Evaluation of Flan-T5 from Hugging Face 

In [18]:
from transformers import pipeline

# Load a pre-trained model (Flan-T5)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

# Function to evaluate model accuracy
def evaluate_model(model, dataset):
    correct = 0
    total = len(dataset)

    for item in dataset:
        question = item["question"]
        correct_answer = item["answer"]

        # Zero-shot prediction
        prompt = f"Give me the final answer without any explaination, just the couple of words that directly show the answer for the Question: {question} with Choices: {', '.join(item['choices'])} Answer:"
        prediction = model(prompt, max_length=20, truncation=True)[0]["generated_text"]

        if correct_answer in prediction:
            correct += 1

    accuracy = (correct / total) * 100
    return accuracy

# Check pre-trained model's accuracy before fine-tuning
zero_shot_accuracy = evaluate_model(qa_pipeline, test_data[:1])
print(f"Zero-Shot Accuracy: {zero_shot_accuracy:.2f}%")


ValueError: Could not load model google/flan-t5-large with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>). See the original errors:

while loading with AutoModelForSeq2SeqLM, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\mariy\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\pipelines\base.py", line 290, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "c:\Users\mariy\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\models\auto\auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs, **kwargs
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  Fil